# Util Bloqs

The utility bloqs let you reshape Soquets.

In [ ]:
from qualtran.drawing import GraphDrawer, PrettyGraphDrawer
from qualtran.bloqs.util_bloqs import Split, Join, Partition
import numpy as np

from IPython.display import SVG

def show_bloq(bloq, draw_cls=PrettyGraphDrawer):
    display(SVG(draw_cls(bloq).get_graph().create_svg()))

## Split and Join

Split and join take a `bitsize=n, shape=()` to `bitsize=1, shape=(n,)` and back.

In [ ]:
show_bloq(Split(3))

In [ ]:
show_bloq(Join(3))

## SplitJoin

For fun, we can pair `Split` and `Join` into an identity operation.

In [ ]:
from attrs import frozen
from functools import cached_property
from typing import *
from qualtran import Bloq, Signature, Register
from qualtran.bloqs.basic_gates import CNOT

@frozen
class SplitJoin(Bloq):
    n: int

    @cached_property
    def signature(self) -> Signature:
        return Signature([Register('x', self.n)])

    def build_composite_bloq(
        self, bb: 'BloqBuilder', *, x: 'Soquet'
    ) -> Dict[str, 'Soquet']:
        xs = bb.split(x)
        xs[0], xs[1] = bb.add(CNOT(), ctrl=xs[0], target=xs[1])
        x = bb.join(xs)
        return {'x': x}

In [ ]:
show_bloq(SplitJoin(10))

In [ ]:
show_bloq(SplitJoin(10).decompose_bloq())

## Partition

A Partition is a useful bloq for abstracting away the details of large registers (like multiple selection registers, spin, orbital, ...). We can use a partition bloq to hide these detailed registers until we decompose the bloq further.

In [ ]:
regs = (Register('xx', 2, shape=(2,3)), Register('yy', 37))
bitsize = sum(reg.total_bits() for reg in regs)
bloq = Partition(n=bitsize, regs=regs)
show_bloq(bloq)

An example of using a `Partition` as part of a decomposition is given below:

In [ ]:
from qualtran import BloqBuilder, Soquet, SoquetT
from qualtran.bloqs.basic_gates import Hadamard, CNOT

@frozen
class ComplicatedBloq(Bloq):
    @cached_property
    def signature(self) -> Signature:
        return Signature([Register('xx', 1), Register('yy', 2, shape=(2, 2)), Register('zz', 3)])

    def build_composite_bloq(
        self, bb: 'BloqBuilder', xx: 'SoquetT', yy: 'SoquetT', zz: 'SoquetT'
    ) -> Dict[str, 'Soquet']:
        xx = bb.add(Hadamard(), q=xx)
        for i in range(2):
            for j in range(2):
                a, b = bb.split(yy[i, j])
                a, b = bb.add(CNOT(), ctrl=a, target=b)
                yy[i, j] = bb.join(np.array([a, b]))
        a, b, c = bb.split(zz)
        zz = bb.join(np.array([a, b, c]))
        return {'xx': xx, 'yy': yy, 'zz': zz}

    def short_name(self) -> str:
        return 'CB'

@frozen
class BlackBoxBloq(Bloq):
    test_bloq: Bloq

    @cached_property
    def bitsize(self):
        return sum(reg.total_bits() for reg in self.test_bloq.signature)

    def short_name(self) -> str:
        return "BBBloq" 

    @cached_property
    def signature(self) -> Signature:
        return Signature.build(test_regs=self.bitsize)

    def build_composite_bloq(self, bb: 'BloqBuilder', test_regs: 'SoquetT') -> Dict[str, 'Soquet']:
        bloq_regs = self.test_bloq.signature
        partition = Partition(self.bitsize, bloq_regs)
        out_regs = bb.add(partition, x=test_regs)
        out_regs = bb.add(self.test_bloq, **{reg.name: sp for reg, sp in zip(bloq_regs, out_regs)})
        test_regs = bb.add(
            partition.dagger(), **{reg.name: sp for reg, sp in zip(bloq_regs, out_regs)}
        )
        return {'test_regs': test_regs}

In [ ]:
show_bloq(BlackBoxBloq(ComplicatedBloq()))

In [ ]:
show_bloq(BlackBoxBloq(ComplicatedBloq()).decompose_bloq())